In [7]:
# Import libraries
import matplotlib.pyplot as plt




hello


In [8]:
# Node Class
class Node ():
    def __init__(self, parent = None, position = None):
        self.parent = parent
        self.position = position
        
        self.g = 0
        self.h = 0
        self.f = 0
    
    def __eq__(self, other):
        return self.position == other.position

In [9]:
# A Star Algorithm

def astar(maze, start, end):
    STEP_COST = 1
    
    start_node = Node(None, start)
    start_node.g = start_node.f = start.node.h = 0
    
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0
    
    open_list = []
    closed_list = []
    
    open_list.append(start_node)
    
    # start the searching loop
    while len(open_list) > 0:
        # get the current node (with the least f score)
        current_node = open_list[0]
        current_index = 0
        
        for index, node in enumerate(open_list):
            if node.f < current_node.f:
                current_node = node
                current_index = index
        
        # pop the current node out of the open list and add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)
        
        # check if the current node is the goal
        if current_node == end_node:
            path = [] # contain the path from start to end node
            
            while current is not None:
                path.append(current.position)
                current = current.parent
            
            return path[::-1]
        
        # Generate neighbor nodes from current node
        neighbors = []
        
        for new_poisition in [(0, 1),(0, -1),(1, 0),(-1, 0)]:
            node_position = (current_node.position[0] + new_poisition[0], current_node.position[1] + new_poisition[1])
            
            # check if the new node is out of bound
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue
            
            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            neighbors.append(new_node)

        # Loop through children
        for neighbor in neighbors:

            # Create the f, g, and h values
            neighbor.g = current_node.g + STEP_COST
            neighbor.h = (((neighbor.position[0] - end_node.position[0]) ** 2) + ((neighbor.position[1] - end_node.position[1]) ** 2))**0.5 # qEdit
            neighbor.f = neighbor.g + neighbor.h

            # Update f(n)
            for index, item in enumerate(open_list):
                if neighbor.position == item.position and neighbor.f < item.f:
                    open_list.pop(index) # Remove the node to replace with child later   
                    
                    # Add the child to the open list (update new child with new f(n))
                    open_list.append(neighbor)